In [16]:
import torch 
import torch.nn as nn 
import pandas as pd 
import plotly.graph_objects as go 

In [75]:
class LogisticRegression:


    def __init__(self ,lower_0, upper_0, sample_size_0, noise_0,lower_1, upper_1, sample_size_1, noise_1):

        # Parameters
        self.lower_0 = lower_0
        self.upper_0 = upper_0
        self.sample_size_0 = sample_size_0
        self.noise_0 = noise_0
        self.lower_1 = lower_1
        self.upper_1 = upper_1
        self.sample_size_1 = sample_size_1
        self.noise_1 = noise_1

        # made by Paramters
        self.x0 = torch.linspace(lower_0, upper_0, sample_size_0) + torch.tensor([noise_0])
        self.x1 = torch.linspace(lower_1, upper_1, sample_size_1) - torch.tensor([noise_1])
        self.X = torch.cat((self.x0, self.x1), dim=0)
        self.y = torch.cat((torch.zeros(len(self.x0)), torch.ones(len(self.x1))), dim=0)

        # stand alone 
        self.inter_and_extrapolation = torch.linspace(-3,3,1000)
        self.possible_weights = torch.linspace(-5,25,100)
        self.loss_fn = nn.BCEWithLogitsLoss()





    def model(self,w):
        self.w = w 
        L = []

        for weight in self.possible_weights:
            z = weight * self.X
            loss = self.loss_fn(z,self.y)
            L.append(loss)

        L = torch.as_tensor(L)
        secret_weight = self.possible_weights[torch.argmin(L)]

        return secret_weight,L
        
        

    def generate_plot(self):
        scatter_class_0 = go.Scatter(
            x=self.x0,
            y=torch.zeros(len(self.x0)),
            mode='markers',
            marker=dict(color='purple'),
            name='class 0'
        )
        scatter_class_1 = go.Scatter(
            x=self.x1,
            y=torch.ones(len(self.x1)),
            mode='markers',
            marker=dict(color='orange'),
            name='class 1'
        )

        non_linear_line = go.Scatter(
            x= self.inter_and_extrapolation,
            y=torch.sigmoid(self.w * self.inter_and_extrapolation),
            mode='lines',
            line={'color': 'rgb(27,158,119)'},
            name='model'
        )
        
        layout = go.Layout(
            xaxis=dict(
                title='X',
                zeroline=True,
                zerolinewidth=2,
                zerolinecolor='rgba(205, 200, 193, 0.7)'
            ),
            yaxis=dict(
                title='Y',
                zeroline=True,
                zerolinewidth=2,
                zerolinecolor='rgba(205, 200, 193, 0.7)'
            ),
 
 
        )
        figure = go.Figure(data=[scatter_class_0, scatter_class_1, non_linear_line], layout=layout)
        return figure
 

    def loss_landscape(self,secret_weight,L):
        
        # landscape
        loss_landscape = go.Scatter(
                x = self.possible_weights,
                y = L,
                mode = 'lines',
                line = dict(color='pink'),
                name ='Loss function landscape'
            )

        # global
        Global_minima = go.Scatter(
            x = (secret_weight,),
            y = (torch.min(L),),
            mode = 'markers',
            marker = dict(color='yellow',size=10,symbol='diamond'),
            name = 'Global minima'
        )


        # ball
        z = self.w * self.X     #forward pass
        loss = self.loss_fn(z,self.y)

        ball = go.Scatter(
                x = (self.w,),
                y = (loss,),
                mode = 'markers',
                marker= dict(color='red'),
                name = 'loss'
        )

        layout = go.Layout(
                xaxis = dict(title='w',
                            range = [-8,25],
                            zeroline = True,
                            zerolinewidth = 2,
                            zerolinecolor = 'rgba(205, 200, 193, 0.7)'),

                yaxis = dict(title='L',
                            range=[0,1.6],
                            zeroline = True,
                            zerolinewidth = 2,
                            zerolinecolor = 'rgba(205, 200, 193, 0.7)')
            )


        figure = go.Figure(data = [loss_landscape,Global_minima,ball],layout=layout)
        
        return figure

In [95]:
data = LogisticRegression(lower_0=-3,upper_0=-1.5,sample_size_0=8,noise_0=1.2,
                          lower_1=1,upper_1=2,sample_size_1=3,noise_1=1.5)

In [99]:
secret_weight,L = data.model(-2)

In [100]:
data.generate_plot()

In [101]:
data.loss_landscape(secret_weight,L)